In [3]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes, Details
from msrest.authentication import CognitiveServicesCredentials

import os 
import cv2
import requests

key = 'f0aee5b0e42d488790ec3b05852f468e'

credentials = CognitiveServicesCredentials(key)
client = ComputerVisionClient(endpoint='https://computer-vision-one.cognitiveservices.azure.com/', credentials=credentials)


In [4]:
capture = cv2.VideoCapture('./resources/videos/top_gun_2.mp4')
fps = capture.get(cv2.CAP_PROP_FPS)
number_of_frames = capture.get(int(cv2.CAP_PROP_FRAME_COUNT))

image_interval = int(number_of_frames/10)

print(image_interval)

count = 0
image_counter = 0 

byte_images = []


while(capture.isOpened()):

    ret, frame = capture.read()
    count = count + 1
    if ret is False:
        break

    if (count==image_interval):
        count = 0
        image_counter = image_counter + 1
        cv2.imwrite(f'./output_frames/image{image_counter}.jpg',frame)
        image_data = open(f'./output_frames/image{image_counter}.jpg', 'rb')

        byte_images.append(image_data)


capture.release()
cv2.destroyAllWindows()


#define what we want to find with azure 
image_features = [VisualFeatureTypes.tags]

def analyse_image(byteimage_list):

    results = []

    for byteimage in byteimage_list:

        result = client.analyze_image_in_stream(byteimage,image_features)

        results.append(result)

        


    return results

##analysing the images 

results = analyse_image(byte_images)




360


In [5]:
for idx, result in enumerate(results):
    print(f"Image {idx}" + "\n")
    for tag in result.tags:
        print(tag)

Image 0

{'additional_properties': {}, 'name': 'clothing', 'confidence': 0.9951426982879639, 'hint': None}
{'additional_properties': {}, 'name': 'human face', 'confidence': 0.9942840933799744, 'hint': None}
{'additional_properties': {}, 'name': 'person', 'confidence': 0.9785916805267334, 'hint': None}
{'additional_properties': {}, 'name': 'man', 'confidence': 0.966691792011261, 'hint': None}
{'additional_properties': {}, 'name': 'screenshot', 'confidence': 0.9661315679550171, 'hint': None}
{'additional_properties': {}, 'name': 'military uniform', 'confidence': 0.8019520044326782, 'hint': None}
Image 1

{'additional_properties': {}, 'name': 'human face', 'confidence': 0.9968690872192383, 'hint': None}
{'additional_properties': {}, 'name': 'screenshot', 'confidence': 0.9672901630401611, 'hint': None}
{'additional_properties': {}, 'name': 'person', 'confidence': 0.9634195566177368, 'hint': None}
{'additional_properties': {}, 'name': 'forehead', 'confidence': 0.9403527975082397, 'hint': No

In [15]:
search_param = 'military uniform'

In [16]:
for idx, result in enumerate(results):
    
    for tag in result.tags:
        if tag.name == search_param:
            print(f"{idx}")
            for tag in result.tags: 
                print(tag)

0
{'additional_properties': {}, 'name': 'clothing', 'confidence': 0.9951426982879639, 'hint': None}
{'additional_properties': {}, 'name': 'human face', 'confidence': 0.9942840933799744, 'hint': None}
{'additional_properties': {}, 'name': 'person', 'confidence': 0.9785916805267334, 'hint': None}
{'additional_properties': {}, 'name': 'man', 'confidence': 0.966691792011261, 'hint': None}
{'additional_properties': {}, 'name': 'screenshot', 'confidence': 0.9661315679550171, 'hint': None}
{'additional_properties': {}, 'name': 'military uniform', 'confidence': 0.8019520044326782, 'hint': None}


In [17]:
for idx, result in enumerate(results):
    
    for tag in result.tags:
        if tag.name == search_param:
            image = cv2.imread(f'./output_frames/image{idx+1}.jpg')
            cv2.imshow('window',image)
            cv2.waitKey(0)
            cv2.destroyAllWindows()